In [54]:
# Gerekli kütüphaneleri yükle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
import joblib

# Performans metriklerini hesaplamak için yardımcı fonksiyonlar
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def r2(y_true, y_pred):
    return r2_score(y_true, y_pred)

# Veri setini yükle
df = pd.read_csv('veri.csv')

# Veri ön işleme
df = pd.get_dummies(df, drop_first=True)
X = df.drop('Fiyat (TL)', axis=1)
y = df['Fiyat (TL)']

# Eğitim ve test setlerine ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ölçeklendirme
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Algoritma listesi
algorithms = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "Support Vector Regressor": SVR(),
    "K-Nearest Neighbors": KNeighborsRegressor(),
    "Ridge": Ridge(),
    "Lasso": Lasso(max_iter=10000),
    "ElasticNet": ElasticNet(max_iter=10000)
}

# Hyperparameter Tuning için GridSearchCV parametreleri
param_grids = {
    "Random Forest": {
        'n_estimators': [100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5]
    },
    "Gradient Boosting": {
        'n_estimators': [100, 200],
        'learning_rate': [0.1, 0.05],
        'max_depth': [3, 4]
    },
    # Diğer algoritmalar için de benzer parametre grupları eklenebilir
}

# Algoritmaları eğit ve değerlendir
best_model = None
best_score = float('-inf')  # R² için en yüksek değeri arıyoruz
results = {}

for name, model in algorithms.items():
    if name in param_grids:
        grid = GridSearchCV(model, param_grids[name], cv=3, scoring='r2')
        grid.fit(X_train, y_train)
        model = grid.best_estimator_
    else:
        model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    score = r2(y_test, y_pred)
    results[name] = score
    if score > best_score:
        best_score = score
        best_model = model

# En iyi modeli kaydet
joblib.dump(best_model, 'eniyi.joblib')

# Sonuçları yazdır
print("Algoritma Performansları (R²):")
for name, score in results.items():
    print(f"{name}: {score}")

print(f"\nEn iyi model: {type(best_model).__name__} - R²: {best_score}")
print("En iyi model 'eniyi.joblib' dosyasına kaydedildi.")


Algoritma Performansları (R²):
Linear Regression: -0.09131887400776395
Decision Tree: -1.0192689802532948
Random Forest: -0.07662750881994906
Gradient Boosting: -0.0992940174647492
Support Vector Regressor: -5.6511456695051976e-05
K-Nearest Neighbors: -0.21711295814748222
Ridge: -0.06668221454889744
Lasso: -0.06696735998493386
ElasticNet: -0.03526706542006841

En iyi model: SVR - R²: -5.6511456695051976e-05
En iyi model 'eniyi.joblib' dosyasına kaydedildi.
